In [ ]:
import math
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
from scipy import stats

df = pd.read_csv('../data/000015', index_col= 'Date', names=['Date', 'Open', 'Close', 'High', 'Low', 'Volume', 'Money', 'PE', 'PB'], parse_dates=True, header=None)

df['Return'] = df['Close'].pct_change() * 100

# Converting daily returns to monthly returns

In [ ]:
df_monthly = pd.DataFrame(df['Close'].asfreq('M', method='ffill'))

df_monthly['Return'] = df_monthly.pct_change() * 100
df_monthly

# Converting daily returns to quarterly returns

In [ ]:
df_quarterly = pd.DataFrame(df['Close'].asfreq('Q', method='ffill'))

df_quarterly['Return'] = df_quarterly.pct_change() * 100
df_quarterly

# Converting daily returns to annual returns

In [ ]:
df_annual = pd.DataFrame(df['Close'].asfreq('A', method='ffill'))

df_annual['Return'] = df_annual.pct_change() * 100
df_annual